# Comparando estratégias

- Amostra por incerteza
- Amostragem aleatória
- Consulta por comitê
- Aprendizado passivo
- Redução do erro esperado

In [80]:
import numpy as np
import pandas as pd
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [17]:
# Plots
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [18]:
from copy import deepcopy

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling

In [21]:
from timeit import default_timer as timer

In [22]:
from sklearn.model_selection import ShuffleSplit

## Classificadores

### Algoritmos

In [23]:
from sklearn.neighbors import KNeighborsClassifier

### Conjunto de dados

In [24]:
from sklearn.datasets import load_iris, load_digits, load_wine, load_breast_cancer

## Amostra por incerteza

In [33]:
def uncertain_sampling(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, n_queries):
    
    from modAL.uncertainty import classifier_uncertainty
    sample_size = 0 #contador de amostras utilizadas pela estratégia
    performance_history = []
    start = timer()
    
    # parte randomica inicial da estratégia
    initial_idx = np.random.choice(range(len(idx_data[idx_bag][0])), size=init_size, replace=False)
    X_train, y_train = X_raw[idx_data[idx_bag][0][initial_idx]], y_raw[idx_data[idx_bag][0][initial_idx]]
    X_test, y_test = X_raw[idx_data[idx_bag][1]], y_raw[idx_data[idx_bag][1]]
    
    sample_size = sample_size + len(X_train)

    classifier.fit(X_train,y_train)

    learner = ActiveLearner (
        estimator=classifier,
        query_strategy=uncertainty_sampling
    )
    uncertain_sample_score = learner.score(X_test, y_test)

    performance_history.append(uncertain_sample_score)

    total_of_samples = 0
    while (total_of_samples != n_queries - 1):
        
        #X_train, X_test, y_train, y_test = train_test_split(X_raw, y_raw, train_size=0.03)
        
        idx = np.random.choice(range(len(idx_data[idx_bag][0])), size=init_size, replace=False)
        X_train, y_train = X_raw[idx_data[idx_bag][0][idx]], y_raw[idx_data[idx_bag][0][idx]]
        #print("WHILE", classifier_uncertainty(learner, X_train[0].reshape(1,-1)))
        if classifier_uncertainty(learner, X_train[0].reshape(1,-1)) > 0.2: #ASK04
            #print("IF", learner.score(X_test, y_test))
            sample_size = sample_size + len(X_train)
            learner.teach(X_train, y_train)
            uncertain_sample_score = learner.score(X_test, y_test)
            performance_history.append(uncertain_sample_score)
            #print("Uncertainty Sampling score: ", uncertain_sample_score)
            performance_history.append(uncertain_sample_score)    
        total_of_samples = total_of_samples + 1 #ASK05
    
    end = timer()
    time_elapsed = end - start
    
    return { "performance_history": performance_history, 
             "time_elapsed": time_elapsed,
             "sample_size": sample_size, # RETORNAR TODAS AS AMOSTRAS DE CADA PERFORMANCE OU SÓ DO ULTIMO
             "Strategy": "Uncertainty Sampling"}

## Amostragem aleatória

In [59]:
def random_sampling(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, n_queries):
        
    sample_size = 0 #contador de amostras utilizadas pela estratégia
    performance_history = []
    start = timer()

    for i in range(1,n_queries+1):

        #high = X_raw.shape[0] = qtd amostras no dataset
        training_indices = np.random.randint(low=0, high=len(X_raw[idx_data[idx_bag][0]]), size=k+i) #high = qtd elementos na bag
        sample_size = sample_size + len(training_indices)

        X_train = X_raw[idx_data[idx_bag][0][training_indices]] #ASK06
        y_train = y_raw[idx_data[idx_bag][0][training_indices]]

        X_test = np.delete(X_raw, idx_data[idx_bag][0][training_indices], axis=0)
        y_test = np.delete(y_raw, idx_data[idx_bag][0][training_indices], axis=0)

        classifier.fit(X_train, y_train)

        #pred = classifier.predict(X_test)
        #print("Random Sampling score: ", knn.score(X_test,y_test))
        performance_history.append(classifier.score(X_test,y_test))

        
    end = timer()
    time_elapsed = end - start

    return { "performance_history": performance_history, 
         "time_elapsed": time_elapsed,
         "sample_size": sample_size, # RETORNAR TODAS AS AMOSTRAS DE CADA PERFORMANCE OU SÓ DO ULTIMO
         "Strategy": "Random Sampling"}

## Consulta por comitê

In [74]:
def query_by_committee(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, n_queries):

    from modAL.models import ActiveLearner, Committee
    from modAL.disagreement import vote_entropy_sampling

    sample_size = 0 #contador de amostras utilizadas pela estratégia
    performance_history = []
    start = timer()


    for i in range(1,n_queries+1):

        learner_list = []

        for j in range(1,n_queries+1): # Loop para criação do comitê

            X_train, _, y_train, _ = train_test_split(X_raw[idx_data[idx_bag][0]], y_raw[idx_data[idx_bag][0]], train_size=0.03)
            sample_size = sample_size + len(X_train)
            
            # initializing learner
            learner = ActiveLearner(
                estimator=which_classifier(init_size),
                query_strategy=uncertainty_sampling,
                X_training = X_train, y_training = y_train 
            )
            learner_list.append(learner)

        # assembling the committee
        committee = Committee(
            learner_list=learner_list,
            query_strategy=vote_entropy_sampling)

        X_pool, y_pool = X_raw[idx_data[idx_bag][0]], y_raw[idx_data[idx_bag][0]]

        # query by committee
        for idx in range(n_queries):
            query_idx, query_instance = committee.query(X_pool, n_instances = k+1)
            sample_size = sample_size + len(query_idx)
            committee.teach(
                X=X_pool[query_idx],
                y=y_pool[query_idx]
            )

            X_pool = np.delete(X_pool, query_idx, axis=0)
            y_pool = np.delete(y_pool, query_idx)
        performance_history.append(committee.score(X_pool, y_pool))

        #print(idx, n_queries, "Query by Committee: ", committee.score(X_pool, y_pool))

        
    end = timer()
    time_elapsed = end - start

    return { "performance_history": performance_history, 
         "time_elapsed": time_elapsed,
         "sample_size": sample_size, # RETORNAR TODAS AS AMOSTRAS DE CADA PERFORMANCE OU SÓ DO ULTIMO
         "Strategy": "Query by Committee"}

### Setup

In [28]:
def which_dataset(dataset = "iris", n_splits = 5):
    
    # Futuramente essa etapa será ajustada para receber qualquer dataset (ou lista com datasets)
    if (dataset == "iris"):
        data = load_iris()
        X_raw = data['data']
        y_raw = data['target']
        
    # cross validation bags
    data_cv = ShuffleSplit(n_splits= n_splits, test_size=0.3, random_state=0) #n_splits
    
    # extraindo ids do data_cv
    idx_data = []
    for train_index, test_index in data_cv.split(X_raw):
            idx_data.append([train_index, test_index])

    return X_raw, y_raw, idx_data

In [29]:
def which_classifier(parameters, classifier = 'knn'):
    
    if (classifier == 'knn'):
        return KNeighborsClassifier(parameters)

In [77]:
n_queries = 10 # cost
n_splits = 5 # number of bags cv
total_performance_history = []
k = 3 #parameter to knn classifier

idx_bag = 1

classifier = which_classifier(k)
X_raw, y_raw, idx_data = which_dataset()

#para cada i em idx_bag ("n_splits") (1 a 5)
for idx_bag in range(n_splits):
    total_performance_history.append(uncertain_sampling(deepcopy(X_raw), deepcopy(y_raw), idx_data, idx_bag, classifier, k, n_queries))
for idx_bag in range(n_splits):
    total_performance_history.append(random_sampling(deepcopy(X_raw), deepcopy(y_raw), idx_data, idx_bag, classifier, k, n_queries))
for idx_bag in range(n_splits):
    total_performance_history.append(query_by_committee(deepcopy(X_raw), deepcopy(y_raw), idx_data, idx_bag, classifier, k, n_queries))

In [78]:
total_performance_history

[{'performance_history': [0.35555555555555557,
   0.35555555555555557,
   0.35555555555555557,
   0.8,
   0.8,
   0.9111111111111111,
   0.9111111111111111,
   0.9111111111111111,
   0.9111111111111111],
  'time_elapsed': 0.06632492300013837,
  'sample_size': 15,
  'Strategy': 'Uncertainty Sampling'},
 {'performance_history': [0.26666666666666666,
   0.37777777777777777,
   0.37777777777777777,
   0.6444444444444445,
   0.6444444444444445,
   0.6444444444444445,
   0.6444444444444445,
   0.8444444444444444,
   0.8444444444444444],
  'time_elapsed': 0.04442180600017309,
  'sample_size': 15,
  'Strategy': 'Uncertainty Sampling'},
 {'performance_history': [0.26666666666666666,
   0.35555555555555557,
   0.35555555555555557,
   0.6222222222222222,
   0.6222222222222222,
   0.6222222222222222,
   0.6222222222222222,
   0.9111111111111111,
   0.9111111111111111],
  'time_elapsed': 0.039439699000467954,
  'sample_size': 15,
  'Strategy': 'Uncertainty Sampling'},
 {'performance_history': [0.26

In [82]:
df = pd.DataFrame.from_dict(total_performance_history)

In [91]:
df.explode('performance_history')

,performance_history,time_elapsed,sample_size,Strategy
0,0.355556,0.066325,15,Uncertainty Sampling
0,0.355556,0.066325,15,Uncertainty Sampling
0,0.355556,0.066325,15,Uncertainty Sampling
0,0.8,0.066325,15,Uncertainty Sampling
0,0.8,0.066325,15,Uncertainty Sampling
...,...,...,...,...
14,0.938462,6.793912,700,Query by Committee
14,0.984615,6.793912,700,Query by Committee
14,0.953846,6.793912,700,Query by Committee
14,0.953846,6.793912,700,Query by Committee


In [ ]:
#  https://seaborn.pydata.org/examples/scatterplot_sizes.html